In [26]:
%load_ext autoreload
%autoreload 2
import sys
# map folder using sys
sys.path.append('../')
import pandas as pd
from galgoz import DATA_FOLDER
from galgoz import Galgoz
from scipy import signal
import numpy as np
from plotly import graph_objects as go
from galgoz.indicators.trend import sg
from whittaker_eilers import WhittakerSmoother
from plotly.subplots import make_subplots
import talib


gz=Galgoz()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Prepare the data for analysis

In [27]:
filename = DATA_FOLDER / "GBP_JPY_H1.pkl"
data = pd.read_pickle(filename)
data["time"] = pd.to_datetime(data["time"])
data["time_str"] = data["time"].dt.strftime(" %-b %d, '%y %H:%M")
data[["mid_o", "mid_h", "mid_l", "mid_c"]] = data[["mid_o", "mid_h", "mid_l", "mid_c"]].astype(float)
data.set_index("time", inplace=True)

#### Use a fraction of the data to run indicators

In [28]:
df_an = data.iloc[-500:].copy()
sg_trend = sg(df_an["mid_c"])

# Money Flow Index
mfi = talib.MFI(
    df_an["mid_h"].values, 
    df_an["mid_l"].values,
    df_an["mid_c"].values,
    df_an["volume"].values,
    timeperiod=14)

# Covert to pandas.Series
mfi = pd.Series(mfi, index=df_an.index)

## Prepare list of indicators and plot

In [32]:
indicators = [{"name": "Savitzy-Golay",
               "data": sg_trend,
               "mode": "lines",
               "row": 1,
               "line": {"color": "black", "width": 2},
               "marker": dict()
               },
              {"name": "MFI",
               "data": mfi,
               "mode": "lines+markers",
               "row": 2,
               "line": {"color": "blue", "width": 2},
               "marker": dict(
                   size=5,  # Size of the markers
                   color='red',  # Color of the markers
                   symbol='circle')
               }]

fig = gz.plot_candles(df_an, instrument=gz.instrument, indicators=indicators, show=True, width=1600, height=800)

In [33]:
type(sg_trend.index[0])

pandas._libs.tslibs.timestamps.Timestamp